Proyecto Datos estadisticos de Bancos BCParaguay; Data Engineer:  Rodrigo Figueredo
Objetivo: se necesita un proceso consistente de extraccion de datos para cargarlo en un Datawarehouse en SQL, el proceso deberá correr de forma mensual y que contenga los datos consistente y evitar a manipulación manual del excel origen.

🧩1. Extracción (Extract)

In [1]:
## Importamos las libreria necesarias para el Proyecto
import pandas as pd

# Cargar todas las hojas del Excel en un diccionario
excel_file = 'C:/Users/rfigu/Documents/Python Scripts/BoletinBCP/Tablas_Boletin_Bancos_Jul25.xlsx'
sheets_dict = pd.read_excel(excel_file, sheet_name=None)

# Acceder a cada hoja como DataFrame
for sheet_name, df in sheets_dict.items():
    print(f"Hoja: {sheet_name}, Filas: {len(df)}")

Hoja: EEFF, Filas: 224012
Hoja: Ratios, Filas: 82673
Hoja: Carteras, Filas: 49860
Hoja: Credito Sector, Filas: 43629
Hoja: CreditoActividad, Filas: 9333
Hoja: Categoría_Creditos, Filas: 14500
Hoja: TC, Filas: 3872
Hoja: Canales_Person, Filas: 13198
Hoja: Inhab, Filas: 7443


In [23]:
## Vista previa de los datos la hoja
sheets_dict.get('Carteras').head()

,Fecha,Codigo Entidad,Codigo Cuenta,Codigo Moneda,Importe
0,2016-01-31,1002,Cartera Vigente,6200,2.383314e+05
1,2016-01-31,1002,Cartera Vigente,6900,2.075860e+06
2,2016-01-31,1002,Cartera Vigente Neta de Previsiones,6200,2.383314e+05
3,2016-01-31,1002,Cartera Vigente Neta de Previsiones,6900,2.051322e+06
4,2016-01-31,1002,Renovados,6200,1.601913e+04


In [26]:
## Vista previa de los tipos de datos la hoja
sheets_dict.get('TC').dtypes

Fecha             datetime64[ns]
Codigo Entidad             int64
Clasificación             object
Total                    float64
dtype: object